# Summary

Notebook to run quick starts and to test code


In [1]:
import os, json


from langchain_openai import ChatOpenAI
# 
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
cred_path = "../../../SocialScrapers/data/environment"
cred_file = "api_creds.json"

with open(os.path.join(cred_path, cred_file), 'r') as file:
    creds = json.load(file)
    
os.environ["OPENAI_API_KEY"] = creds["open_api_key"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = creds["langsmith_api_key"]



In [3]:
llm = ChatOpenAI(model="gpt-4o-mini")

import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://archive.org/stream/2022californiare00unse/2022californiare00unse_djvu.txt",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            # class_=("post-content", "post-title", "post-header")
            "pre"
        )
        
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is the minimum size for a bathroom in California?")

'The minimum size for a bathroom in California is not explicitly stated; however, bathrooms are limited to a maximum area of 55 square feet (5.1 m²) according to the California Residential Code. Additionally, bathrooms must have a minimum aggregate glazing area in windows of at least 3 square feet, with half of that being openable. Therefore, while there is no specified minimum size, the design must fit within these regulatory parameters.'